In [ ]:
#Importing necessary libraries and setting home paths
from astropy.table import Table
from matplotlib import pyplot as plt
%matplotlib inline
import os
import numpy as np
from astropy.io.ascii import masked
from astropy.coordinates import Angle
from astropy.io import ascii
import glob
from astropy.io import fits
import wget
import matplotlib.image as mpimg
from astropy.wcs import WCS
from scipy.stats import scoreatpercentile
from astropy.visualization import simple_norm
from reproject import reproject_interp
import sys
from IPython.display import clear_output
from photutils.detection import DAOStarFinder
from astropy.stats import sigma_clipped_stats
from photutils.aperture import CircularAperture
from astropy.visualization import SqrtStretch
from astropy.visualization import ImageNormalize
from astropy.visualization import LogStretch
from astropy.wcs import WCS
import astropy.units as u
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import Angle
from scipy import stats
from astropy.visualization import MinMaxInterval
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
from matplotlib import pyplot as plt
from matplotlib import colors
from astropy.stats import gaussian_sigma_to_fwhm
import pandas as pd
from reproject import reproject_interp
from astropy.nddata import CCDData
import warnings
warnings.filterwarnings('ignore')

mycolors = plt.rcParams['axes.prop_cycle'].by_key()['color']

from photutils.isophote import EllipseGeometry
from photutils.aperture import EllipticalAperture

#define an empty dictionary that will contain the EllipseGeometry instance
geometry = {}
initparams = {}
#initialize dictionary for half-light radii
rhalfpix = {}
rhalfasec = {}
from photutils.isophote import Ellipse
from photutils import aperture_photometry
#initialize dictionary for ellipse fitting
ellipse = {}
isolist = {}
import matplotlib.pyplot as plt
from astropy.visualization import simple_norm
mycolors = plt.rcParams['axes.prop_cycle'].by_key()['color']
#Set the home path

os.environ['HOME'] ='C:/Users/USER/Documents/GitHub'
homedir = os.getenv("HOME")
tabledir = homedir+'/Virgo/tables/'
plotdir = homedir+'/Virgo/plots/'
datadir = homedir+'/masking/' #set to where to find the masking data and images, all the contents of the masking here should be in the zip file

In [1]:
from masking_funct import *

In [ ]:
#We need to find the central pixels of the galaxies first, so this program does that
# Input CSV file with galaxy data
csv_file = tabledir + '/Photometrytesting.csv'
galaxy_data = pd.read_csv(csv_file)

# Prepare output CSV
output_data = []

# Process each galaxy
for i in range(len(galaxy_data)):
    galaxy_name = str(galaxy_data['GALAXY'][i])
    RA = galaxy_data['RA_MOMENT'][i]
    DEC = galaxy_data['DEC_MOMENT'][i]
    VFID = f"VFID{int(galaxy_data['VF_ID'][i]):04d}"

    # Paths for the images
    base_path = os.path.join(datadir, 'pipeline', galaxy_name)
    mask_path = os.path.join(datadir, 'masks')
    
    # Search for the HPPUNIMAPB FITS file dynamically
    destination_folder = os.path.join(datadir, 'pipeline', galaxy_name, 'HPPUNIMAPB')
    partial_name = 'hpacs_25HPPUNIMAPB'
    found_files = find_files(destination_folder, partial_name)
    hppunimapb_image_path = found_files[0] if found_files else None

    # Search for the HPPUNIMAPG FITS file dynamically
    destination_folder = os.path.join(datadir, 'pipeline', galaxy_name, 'HPPUNIMAPG')
    partial_name = 'hpacs_25HPPUNIMAPB'
    found_files = find_files(destination_folder, partial_name)
    hppunimapg_image_path = found_files[0] if found_files else None
    # Search for the HPPUNIMAPR FITS file dynamically
    destination_folder = os.path.join(datadir, 'pipeline', galaxy_name, 'HPPUNIMAPR')
    partial_name = 'hpacs_25HPPUNIMAPR'
    found_files = find_files(destination_folder, partial_name)
    hppunimapr_image_path = found_files[0] if found_files else None

    # Check and process WISE mask image
    wise_mask_image_path = os.path.join(mask_path, f'{galaxy_name}-custom-image-wise-mask.fits')
    wise_mask_x, wise_mask_y = None, None
    if os.path.exists(wise_mask_image_path):
        wise_mask_data, wise_mask_header = fits.getdata(wise_mask_image_path, header=True)
        wise_mask_wcs = WCS(wise_mask_header)
        wise_mask_x, wise_mask_y = wise_mask_wcs.all_world2pix(RA, DEC, 0)
    else:
        # If the original WISE mask is not found, check for the alternative '-custom-image-r-mask.fits'
        wise_mask_image_path_r = os.path.join(mask_path, f'{galaxy_name}-custom-image-r-mask.fits')
    
        if os.path.exists(wise_mask_image_path_r):
            wise_mask_data, wise_mask_header = fits.getdata(wise_mask_image_path_r, header=True)
            wise_mask_wcs = WCS(wise_mask_header)
            wise_mask_x, wise_mask_y = wise_mask_wcs.all_world2pix(RA, DEC, 0)
        else:
            # If neither WISE mask is found, set the pixel coordinates to None
            wise_mask_x, wise_mask_y = None, None

    # Initialize pixel coordinates
    hppunimapb_x, hppunimapb_y = None, None
    hppunimapg_x, hppunimapg_y = None, None
    hppunimapr_x, hppunimapr_y = None, None

    # Check and process HPPUNIMAPB image
    if hppunimapb_image_path and os.path.exists(hppunimapb_image_path):
        hppunimapb_data, hppunimapb_header = fits.getdata(hppunimapb_image_path, header=True)
        hppunimapb_wcs = WCS(hppunimapb_header)
        hppunimapb_x, hppunimapb_y = hppunimapb_wcs.all_world2pix(RA, DEC, 0)
    # Check and process HPPUNIMAPB image
    if hppunimapg_image_path and os.path.exists(hppunimapg_image_path):
        hppunimapg_data, hppunimapg_header = fits.getdata(hppunimapg_image_path, header=True)
        hppunimapg_wcs = WCS(hppunimapg_header)
        hppunimapg_x, hppunimapg_y = hppunimapg_wcs.all_world2pix(RA, DEC, 0)
    # Check and process HPPUNIMAPB image
    if hppunimapr_image_path and os.path.exists(hppunimapr_image_path):
        hppunimapr_data, hppunimapr_header = fits.getdata(hppunimapr_image_path, header=True)
        hppunimapr_wcs = WCS(hppunimapr_header)
        hppunimapr_x, hppunimapr_y = hppunimapr_wcs.all_world2pix(RA, DEC, 0)

    # Append the results to the output data
    output_data.append({
        'VFID': VFID,
        'Galaxy': galaxy_name,
        'RA': RA,
        'DEC': DEC,
        'HPPUNIMAPB_X': hppunimapb_x,
        'HPPUNIMAPB_Y': hppunimapb_y,
        'HPPUNIMAPG_X': hppunimapg_x,
        'HPPUNIMAPG_Y': hppunimapg_y,
        'HPPUNIMAPR_X': hppunimapr_x,
        'HPPUNIMAPR_Y': hppunimapr_y,
        'WISE_Mask_X': wise_mask_x,
        'WISE_Mask_Y': wise_mask_y
    })

# Convert to DataFrame and save to CSV
output_df = pd.DataFrame(output_data)
output_csv_path = datadir + '/pixel_coordinates.csv'
output_df.to_csv(output_csv_path, index=False)
print('done')

In [ ]:
#imposing the masks onto the images by lining up the central mask pixel with the central galaxy pixels calculated from the code block earlier
import os
import pandas as pd
from astropy.io import fits
from astropy.table import Table

# File paths
csv_file = os.path.join(datadir, 'pixel_coordinates.csv')
galaxy = Table.read(csv_file)

for i in range(len(galaxy)):
    galaxy_name = str(galaxy['Galaxy'][i])
    path = os.path.join(datadir, 'pipeline', galaxy_name)
    VFID = str(galaxy['VFID'][i])

    if os.path.exists(path):
        for color in ['B', 'G', 'R']:  # Loop over the three color channels
            destination_folder = os.path.join(path, f'HPPUNIMAP{color}')
            if color in ['B', 'G']:  # Blue and Green bands
                partial_name = f'hpacs_25HPPUNIMAPB'
            else:  # Red band
                partial_name = f'hpacs_25HPPUNIMAPR'
            found_files = find_files(destination_folder, partial_name)

            if found_files:
                found_file = found_files[0]

                # Mask file paths
                wise_mask_file = os.path.join(datadir, 'masks', f'{galaxy_name}-custom-image-wise-mask.fits')
                r_mask_file = os.path.join(datadir, 'masks', f'{galaxy_name}-custom-image-r-mask.fits')

                # Output FITS file path
                output_fits = os.path.join(datadir, 'masked', f'{galaxy_name}_masked{color}.fits') #where the masked file should go

                # Read the CSV file for the central pixel coordinates
                coords_data = pd.read_csv(csv_file)
                coords_x = coords_data[f'HPPUNIMAP{color}_X'][i]
                coords_y = coords_data[f'HPPUNIMAP{color}_Y'][i]

                # Check if the wise-mask file exists, otherwise check for the r-mask
                if os.path.exists(wise_mask_file):
                    # If wise-mask exists, overlay it
                    overlay_mask_on_fits(found_file, wise_mask_file, csv_file, output_fits, i, coords_x, coords_y)
                elif os.path.exists(r_mask_file):
                    # If r-mask exists, overlay it
                    overlay_mask_on_fits(found_file, r_mask_file, csv_file, output_fits, i, coords_x, coords_y)
                else:
                    # If no mask files exist, simply copy the original FITS file to the output location
                    if not os.path.exists(os.path.dirname(output_fits)):
                        os.makedirs(os.path.dirname(output_fits))
                    with fits.open(found_file) as hdul:
                        hdul.writeto(output_fits, overwrite=True)
                    print(f"Mask file not found for {galaxy_name} or {VFID}. Saved original FITS as {output_fits}")
